In [113]:
import folium, requests
import pandas as pd
import random

df_gist = pd.read_csv('./gist.csv')
app_key = "l7xx19b0512ba5fd4587992aabcae1c10d9b"

# folium map visualize, center: gist
gist_lat = '35.22956444643729'
gist_lon = '126.84774671246136'

m = folium.Map(location=[gist_lat, gist_lon],
                zoom_start=15,
                width='100%',
                height='100%',
                )

# latitude, logitude of hub and shops
locations = []
locations.append((gist_lat, gist_lon))

# amount of orders for each shops
orders = [random.randint(0, 20) for i in range(len(df_gist))]
orders[0] = 0   # 0 for depot

In [115]:
## skip!!!
# geocoding test code

url = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo?version=1&format=json&callback=result"


r = requests.get(url, params={
        "appKey" : "l7xx19b0512ba5fd4587992aabcae1c10d9b",
        "coordType" : "WGS84GEO",
        "fullAddr": "광주광역시 광산구 월계동831-6번지 1층"
    })

res = r.json().get('coordinateInfo').get('coordinate')[0]

print(res)

{'matchFlag': '', 'lat': '', 'lon': '', 'latEntr': '', 'lonEntr': '', 'city_do': '광주', 'gu_gun': '광산구', 'eup_myun': '', 'legalDong': '', 'legalDongCode': '', 'adminDong': '', 'adminDongCode': '', 'ri': '', 'bunji': '', 'buildingName': '', 'buildingDong': '', 'newMatchFlag': 'N55', 'newLat': '35.211563', 'newLon': '126.839821', 'newLatEntr': '35.211563', 'newLonEntr': '126.839821', 'newRoadName': '첨단중앙로67번길', 'newBuildingIndex': '', 'newBuildingName': '', 'newBuildingCateName': '', 'newBuildingDong': '', 'zipcode': '62267', 'remainder': ''}


In [116]:
# tmap geocoding api 호출
url = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo?version=1&format=json&callback=result"

for index, row in df_gist.iterrows():

    shop_name = row["name"]
    shop_addr = row["address"]

    r = requests.get(url, params={
        "appKey" : app_key,
        "coordType" : "WGS84GEO",
        "fullAddr": shop_addr
    })

    res = r.json().get('coordinateInfo').get('coordinate')[0]
    
    latitude = ''
    longitude = ''
    
    if (res.get('latEntr') != ''):              # 구주소 입구점 위*경도
        latitude = res.get('latEntr')
        longitude = res.get('lonEntr')
    elif (res.get('newLatEntr') != ''):         # 새주소 입구점 위*경도
        latitude = res.get('newLatEntr')
        longitude = res.get('newLonEntr')
    elif (res.get('lat') != ''):                # 구주소 위*경도
        latitude = res.get('lat')
        longitude = res.get('lon')
    elif (res.get('newLat') != ''):             # 새주소 위*경도
        latitude = res.get('newLat')
        longitude = res.get('newLon')
    else:
        continue

    locations.append((latitude, longitude))
    print(shop_name, shop_addr, latitude, longitude)

    folium.Marker([latitude, longitude],
                popup=shop_name,
                tooltip=shop_name).add_to(m)

print(locations)
m


한솥도시락 광주광역시 광산구 임방울대로 727-6 35.218452 126.837765
맘스터치 광주광역시 광산구 첨단중앙로 96 35.213924 126.843209
킹스샌드&샐러드 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
달콤 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
마마죽&스프 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
김밥랜드 광주광역시 광산구 월계동 758-4 35.221201 126.840571
무양식탁 광주광역시 광산구 첨단내촌로36번길, 28 35.210008 126.838904
2학식당 광주광역시 북구 첨단과기로 123 35.225034 126.840154
또아식빵 광주광역시 광산구 첨단중앙로 106번길 23 35.214924 126.844237
기가찬국밥 광주광역시 광산구 첨단중앙로 106번길 30 35.21473 126.844765
원할머니보쌈 광주광역시 광산구 첨단중앙로182번길 24 35.221757 126.84407
유생촌 광주광역시 광산구 월계동 881-1 35.216063 126.848265
이마트24 광주광역시 광산구 첨단중앙로 96 35.213924 126.843209
통큰고기 통큰비빔밥 광주광역시 광산구 월계동826-3 35.212924 126.841654
마초떡볶이 광주광역시 광산구 월계동 851-16 35.20898 126.841237
일계양득 광주광역시 광산구 월계동 851-16 35.20898 126.841237
국이랑 찌개랑 광주광역시 광산구 월계동831-6번지 1층 35.211563 126.839821
도시락데이 광주광역시 광산구 월계동831-6번지 1층 35.211563 126.839821
김밥나라 광주광역시 광산구 임방울대로 826번길 35.215146 126.848515
설빙 광주광역시 광산구 첨단중앙로 104 2층 35.214758 126.843571
김정호 깐풍기 광주광역시 광산구 산월로 33 35.20

In [152]:
import json

# Tmap 타임머신 자동차 길 안내 api
# 패턴 교통정보를 이용하여 미래의 시간에 차량의 경로를 탐색

url = "https://apis.openapi.sk.com/tmap/routes/prediction?version=1&format=json&totalValue=2"
prediction_time = "2022-06-30T11:30:00+0900"
departure_addr = ["126.98261605850641", "37.57528380389962"]
dest_addr = ["126.98773907773705", "37.56611469775449"]

headers = {
	"appKey" : app_key,
	"Content-Type" : "application/json"
}

datas = {
	"routesInfo" : {
		"departure" : {
			"name" : "test1",
			"lon" : "126.97504418246645",
			"lat" : "37.5700584233685"
		},
		"destination" : {
			"name" : "test2",
			"lon" : "126.997589",
			"lat" : "37.570594"
		},
		"predictionType" : "arrival", # todo. 도착시간 예측 길 안내 -> 출발로 변경?
		"predictionTime" : prediction_time,
		"wayPoints" : {
			"wayPoint" : [ {
				"lon" : departure_addr[0],
				"lat" : departure_addr[1]
			}, {
				"lon" : dest_addr[0],
				"lat" : dest_addr[1]
			} ]
		},
		"searchOption" : "00"   # todo. 우선순위: 교통최적+추천 -> 02.교통최적+최소시간?
	}
}

r = requests.post(url, headers=headers, data=json.dumps(datas))

res = r.json().get('features')[0].get('properties')

pred_distance = res.get('totalDistance')	# meter
pred_time = res.get('totalTime')			# sec

print(pred_distance, pred_time)

4773 1903
